In [1]:
# Classificação de Sentimentos - Tensor Flow

In [27]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [3]:
!tree


.
├── Classificador de Sentimentos.ipynb
├── Datasets
│   ├── NoThemeTweets.csv
│   ├── testdatasets
│   │   ├── Test3classes.csv
│   │   ├── Test.csv
│   │   └── TestTema.csv
│   ├── TestDatasets
│   │   ├── Test3classes.csv
│   │   ├── Test.csv
│   │   └── TestTema.csv
│   ├── trainingdatasets
│   │   ├── Train100.csv
│   │   ├── Train200.csv
│   │   ├── Train300.csv
│   │   ├── Train3Classes.csv
│   │   ├── Train400.csv
│   │   ├── Train500.csv
│   │   ├── Train50.csv
│   │   └── TrainTema.csv
│   ├── TrainingDatasets
│   │   ├── Train100.csv
│   │   ├── Train200.csv
│   │   ├── Train300.csv
│   │   ├── Train3Classes.csv
│   │   ├── Train400.csv
│   │   ├── Train500.csv
│   │   ├── Train50.csv
│   │   └── TrainTema.csv
│   ├── TweetsNeutralHash.csv
│   ├── TweetsNeutralNews.csv
│   └── TweetsWithTheme.csv
├── README.md
└── stopwords.txt

5 directories, 29 files


In [4]:

df_train = pd.read_csv("Datasets/trainingdatasets/Train500.csv",encoding='utf-8', sep=';',engine='python')
df_train.head()
stopword = pd.read_table("stopwords.txt")

In [5]:
df_train.drop(['id','tweet_date','query_used'],axis=1,inplace=True)
df_train.head()

,tweet_text,sentiment
0,@Laranjito76 A pessoa certa para isso seria o ...,1
1,"@behin_d_curtain Para mim, é precisamente o co...",1
2,Vou fazer um video hoje... estou pensando em f...,1
3,"aaaaaaaa amei tanto essas polaroids, nem sei e...",1
4,Valoriza o coração do menininho que vc tem. El...,1


In [12]:
stoplist = []
sentences = []
for i in range(len(stopword)):
    stoplist.append(stopword['stopwords'][i])
    
for i in range(len(df_train['tweet_text'])):
    sentence = df_train['tweet_text'][i]
    for word in stoplist:
        newword = " "+word+" "
        sentence = sentence.replace(newword," ")
        sentence = sentence.replace(newword.upper(),'')
        sentence = sentence.replace("  "," ")
    sentences.append(sentence)
    
#todo remover !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

agora vamos remover os emojis

In [13]:
import re
def emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


In [14]:
sentences = list(map(emoji,sentences))
sentences[0]

In [21]:
training_sentences = []
training_labels = []

training_sentences = sentences
training_labels = df_train['sentiment']

In [22]:
from category_encoders.ordinal import OrdinalEncoder
enc = OrdinalEncoder(cols=['sentiment'])
enc.fit(training_labels)
training_labels_ord = enc.transform(training_labels)

training_labels_ord.head()

,sentiment
0,1
1,1
2,1
3,1
4,1


In [24]:
training_labels = np.array(training_labels_ord)
training_labels

array([[1],
       [1],
       [1],
       ...,
       [2],
       [2],
       [2]])

In [29]:
x_training, x_validation, y_training, y_validation = train_test_split(training_sentences,training_labels,test_size=0.3,random_state=17)
print(len(x_training),len(y_training), len(x_validation), len(y_validation))

350000 350000 150000 150000


In [30]:

vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok,)
tokenizer.fit_on_texts(x_training)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(x_training)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(x_validation)
validation_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [31]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optmizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                12544     
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                1560      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 174,129
Trainable params: 174,129
Non-trainable params: 0
_________________________________________________________________


In [ ]:

num_epochs = 5
history = model.fit(padded, y_training,epochs=num_epochs,validation_data=(validation_padded,y_validation))

Train on 350000 samples, validate on 150000 samples
Epoch 1/5
